In [1]:
import optuna
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np



/Users/kirtan/Documents/calgary-hacks-2025/model/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_height = 180
image_width = 180
batch_size = 32
num_classes = 21  # Update this based on actual classes

# Load Dataset
def get_datasets():
    directory = "../images_train/"
    ds_train, ds_validation = keras.preprocessing.image_dataset_from_directory(
        directory,
        labels='inferred',
        label_mode='int',
        batch_size=batch_size,
        image_size=(image_height, image_width),
        shuffle=True,
        seed=50,
        validation_split=0.20,
        subset="both"
    )
    AUTOTUNE = tf.data.AUTOTUNE
    ds_train = ds_train.cache().shuffle(2200).prefetch(buffer_size=AUTOTUNE)
    ds_validation = ds_validation.cache().prefetch(buffer_size=AUTOTUNE)
    return ds_train, ds_validation

ds_train, ds_validation = get_datasets()



NotFoundError: Could not find directory images_train

In [ ]:
# Optuna Optimization
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    dropout_rate = trial.suggest_float("dropout_rate", 0.3, 0.6)
    filters_1 = trial.suggest_int("filters_1", 16, 64, step=16)
    filters_2 = trial.suggest_int("filters_2", 32, 128, step=32)
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
        tf.keras.layers.Conv2D(filters_1, (4, 4), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(filters_2, (4, 4), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["sparse_categorical_accuracy"]
    )

    history = model.fit(ds_train, validation_data=ds_validation, epochs=10, verbose=0)
    
    return max(history.history["val_sparse_categorical_accuracy"])

# Run Optuna Study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Get Best Parameters
best_params = study.best_params
print("Best Parameters:", best_params)




In [ ]:
# Train Final Model with Best Parameters
final_model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(image_height, image_width, 3)),
    tf.keras.layers.Conv2D(best_params["filters_1"], (4, 4), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(best_params["filters_2"], (4, 4), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(best_params["dropout_rate"]),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

final_model.compile(
    optimizer=tf.keras.optimizers.Adam(best_params["learning_rate"]),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["sparse_categorical_accuracy"]
)

history = final_model.fit(ds_train, epochs=40, validation_data=ds_validation)

# Plot Accuracy
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(["Training", "Validation"])
plt.show()